In [1]:
import os
import pandas as pd
import numpy as np
import random
import dendropy
import treeswift
from treeswift import read_tree_newick
import matplotlib.pyplot as plt
import re

In [2]:
print(os.getcwd())

/Users/nora/Documents/ml_metagenomics/tol_variable_k_resuls/benchmark_runtime_mem


In [3]:
my_files = os.listdir("128G")

In [4]:
my_files[:5]

['Memory Quota-data-2025-02-28 20_43_40.csv',
 'Memory Quota-data-2025-03-03 13_27_40.csv',
 'Memory Quota-data-2025-02-27 12_54_54.csv',
 'Memory Quota-data-2025-02-28 11_03_30.csv',
 'Memory Quota-data-2025-03-01 06_59_41.csv']

In [5]:
my_path = "/Users/nora/Documents/ml_metagenomics"

true_clades = pd.read_csv(os.path.join(my_path, "clade_targets.txt"), sep = " ", header = 0)

In [6]:
true_clade_sizes = {}
for i in range (0, 15):
    true_clade_sizes[i]  = len(true_clades.loc[true_clades["clade"]== i])

In [7]:
true_clade_sizes

{0: 836,
 1: 586,
 2: 846,
 3: 653,
 4: 724,
 5: 507,
 6: 591,
 7: 784,
 8: 803,
 9: 828,
 10: 782,
 11: 736,
 12: 618,
 13: 669,
 14: 612}

In [8]:
columns = ['clade_num','condition', 'claded', 'chunked', 'clade', 'time', 'memory']
df = pd.DataFrame(columns=columns)

In [9]:
df_tmp = pd.DataFrame({'clade_num': range(0, 15)})
df_tmp['condition'] = 'Claded-Chunked'
df_tmp['claded'] = 'Claded'
df_tmp['chunked'] = 'Chunked'


my_dir = 'k7_v58_8k_train_model_10K_TOL_Claded_Chunked_rt'
my_files = os.listdir(my_dir)


# Populate running time
for f in my_files:
    if "train_model" in f:
        print (f)
    
        cur_clade = f.split("clade_")[-1]
        cur_clade_num = int(cur_clade.replace(".log", ""))
        
        file = open(os.path.join(my_dir, f), 'r')
        content = file.read().splitlines()
        t_stamp = content[-1]
        if "Time" in t_stamp:

            print (t_stamp)
            t_stamp_lst = t_stamp.replace("Time: ", "").split(":")
            t_stamp_min = [float(t) for t in t_stamp_lst]
            print (t_stamp_min)
            df_tmp.loc[df_tmp['clade_num']==cur_clade_num, 'time'] = t_stamp_min[0]*60.0 + t_stamp_min[1] + t_stamp_min[2]/60.0
        
        df_tmp.loc[df_tmp['clade_num']==cur_clade_num, 'size'] = int(true_clade_sizes[cur_clade_num])


train_model_20250301_055017_clade_8.log
Time: 05:07:40
[5.0, 7.0, 40.0]
train_model_20250303_212943_clade_2.log
Time: 04:57:29
[4.0, 57.0, 29.0]
train_model_20250303_212936_clade_7.log
Time: 05:38:28
[5.0, 38.0, 28.0]
train_model_20250303_212936_clade_4.log
Time: 04:55:44
[4.0, 55.0, 44.0]
train_model_20250303_212955_clade_12.log
Time: 02:58:38
[2.0, 58.0, 38.0]
train_model_20250301_024632_clade_13.log
Time: 03:30:27
[3.0, 30.0, 27.0]
train_model_20250301_001137_clade_6.log
Time: 04:23:14
[4.0, 23.0, 14.0]
train_model_20250228_232845_clade_5.log
Time: 03:16:00
[3.0, 16.0, 0.0]
train_model_20250301_064835_clade_10.log
Time: 05:42:28
[5.0, 42.0, 28.0]
train_model_20250301_145333_clade_9.log
Time: 04:43:22
[4.0, 43.0, 22.0]
train_model_20250301_102808_clade_1.log
Time: 06:13:40
[6.0, 13.0, 40.0]
train_model_20250303_212943_clade_14.log
Time: 04:10:07
[4.0, 10.0, 7.0]
train_model_20250301_145335_clade_10.log
Time: 04:39:37
[4.0, 39.0, 37.0]
train_model_20250301_082816_clade_9.log
Time: 04:

In [10]:
df_tmp

,clade_num,condition,claded,chunked,time,size
0,0,Claded-Chunked,Claded,Chunked,247.066667,836.0
1,1,Claded-Chunked,Claded,Chunked,373.666667,586.0
2,2,Claded-Chunked,Claded,Chunked,297.483333,846.0
3,3,Claded-Chunked,Claded,Chunked,233.366667,653.0
4,4,Claded-Chunked,Claded,Chunked,295.733333,724.0
5,5,Claded-Chunked,Claded,Chunked,196.000000,507.0
6,6,Claded-Chunked,Claded,Chunked,263.233333,591.0
7,7,Claded-Chunked,Claded,Chunked,338.466667,784.0
8,8,Claded-Chunked,Claded,Chunked,307.666667,803.0
9,9,Claded-Chunked,Claded,Chunked,277.050000,828.0


In [11]:
df_tmp["time"].mean()

270.79

In [12]:
df_tmp["time"].max()

373.6666666666667

In [11]:
# Populate memory
my_dir = '128G'
my_files = os.listdir(my_dir)
print (my_files) 

['Memory Quota-data-2025-02-28 20_43_40.csv', 'Memory Quota-data-2025-03-03 13_27_40.csv', 'Memory Quota-data-2025-02-27 12_54_54.csv', 'Memory Quota-data-2025-02-28 11_03_30.csv', 'Memory Quota-data-2025-03-01 06_59_41.csv', 'Memory Quota-data-2025-02-28 17_37_25.csv', 'Memory Quota-data-2025-02-28 13_47_09.csv', 'Memory Quota-data-2025-03-01 07_22_55.csv', 'Memory Quota-data-2025-02-28 22_42_40.csv', 'Memory Quota-data-2025-02-27 12_58_56.csv', 'Memory Quota-data-2025-02-28 16_24_06.csv', 'Memory Quota-data-2025-02-28 18_06_51.csv', 'Memory Quota-data-2025-02-28 17_13_24.csv', 'Memory Quota-data-2025-03-03 14_54_37.csv', 'Memory Quota-data-2025-02-28 13_42_04.csv', 'Memory Quota-data-2025-02-28 14_21_48.csv', 'Memory Quota-data-2025-03-01 06_58_02.csv', 'Memory Quota-data-2025-02-28 21_17_48.csv', 'Memory Quota-data-2025-03-01 00_00_50.csv', 'Memory Quota-data-2025-02-28 13_57_31.csv', 'Memory Quota-data-2025-02-28 10_39_45.csv', 'Memory Quota-data-2025-03-01 11_22_03.csv', 'Memory Q

In [12]:
for j in range (0, 15):

    cmd = "kf2d-v1-0-59-10ktol-k7-claded-chunked-rt-cl{}-".format(j)
    q = ! grep $cmd 128G/Memory*csv
    print ([w.split(",")[-1] for w in q])
    mem_vals = [float(w.split(",")[-1].split(" ")[0]) for w in q]
    my_max_memory = max(mem_vals)

    df_tmp.loc[df_tmp['clade_num']==j, 'mem'] = my_max_memory
    

['5.37 GiB', '5.38 GiB', '5.39 GiB', '6.01 GiB', '6.01 GiB', '6.12 GiB', '6.12 GiB', '6.13 GiB', '6.09 GiB', '6.13 GiB', '6.10 GiB', '6.13 GiB', '6.13 GiB']
['8.44 GiB', '8.50 GiB', '8.53 GiB', '8.52 GiB', '8.52 GiB', '8.52 GiB', '8.51 GiB', '8.53 GiB', '8.52 GiB', '8.52 GiB', '8.51 GiB', '8.54 GiB', '8.48 GiB', '8.56 GiB', '8.56 GiB', '8.45 GiB', '8.48 GiB', '8.48 GiB', '8.44 GiB', '8.44 GiB', '8.44 GiB', '8.49 GiB', '8.52 GiB', '8.54 GiB']
['2.49 GiB', '3.09 GiB', '7.06 GiB', '7.06 GiB', '9.62 GiB', '7.87 GiB', '7.87 GiB', '7.88 GiB', '7.81 GiB', '7.76 GiB', '7.89 GiB', '7.90 GiB', '7.90 GiB', '7.90 GiB', '7.89 GiB', '7.90 GiB', '7.90 GiB', '7.80 GiB', '7.90 GiB', '7.90 GiB', '7.91 GiB', '7.82 GiB', '7.91 GiB', '7.92 GiB', '7.91 GiB', '7.92 GiB', '7.92 GiB', '7.82 GiB', '7.87 GiB', '7.85 GiB', '7.92 GiB', '7.95 GiB', '7.95 GiB', '7.96 GiB', '7.95 GiB', '7.89 GiB', '7.95 GiB', '7.87 GiB', '7.98 GiB', '8.00 GiB', '8.00 GiB', '8.01 GiB', '8.01 GiB', '7.98 GiB', '8.01 GiB', '8.01 GiB', '

In [13]:
[w.split(",")[-1] for w in q]

['1.44 GiB',
 '7.22 GiB',
 '7.33 GiB',
 '7.33 GiB',
 '7.33 GiB',
 '7.35 GiB',
 '7.25 GiB',
 '7.38 GiB',
 '7.23 GiB',
 '7.23 GiB',
 '7.30 GiB',
 '7.30 GiB',
 '7.30 GiB',
 '7.30 GiB',
 '7.40 GiB',
 '7.42 GiB']

In [14]:
df_tmp

,clade_num,condition,claded,chunked,time,size,mem
0,0,Claded-Chunked,Claded,Chunked,247.066667,836.0,6.13
1,1,Claded-Chunked,Claded,Chunked,373.666667,586.0,8.56
2,2,Claded-Chunked,Claded,Chunked,297.483333,846.0,9.62
3,3,Claded-Chunked,Claded,Chunked,233.366667,653.0,7.65
4,4,Claded-Chunked,Claded,Chunked,295.733333,724.0,8.46
5,5,Claded-Chunked,Claded,Chunked,196.000000,507.0,7.06
6,6,Claded-Chunked,Claded,Chunked,263.233333,591.0,7.57
7,7,Claded-Chunked,Claded,Chunked,338.466667,784.0,9.29
8,8,Claded-Chunked,Claded,Chunked,307.666667,803.0,8.89
9,9,Claded-Chunked,Claded,Chunked,277.050000,828.0,8.58


In [15]:
df_tmp

,clade_num,condition,claded,chunked,time,size,mem
0,0,Claded-Chunked,Claded,Chunked,247.066667,836.0,6.13
1,1,Claded-Chunked,Claded,Chunked,373.666667,586.0,8.56
2,2,Claded-Chunked,Claded,Chunked,297.483333,846.0,9.62
3,3,Claded-Chunked,Claded,Chunked,233.366667,653.0,7.65
4,4,Claded-Chunked,Claded,Chunked,295.733333,724.0,8.46
5,5,Claded-Chunked,Claded,Chunked,196.000000,507.0,7.06
6,6,Claded-Chunked,Claded,Chunked,263.233333,591.0,7.57
7,7,Claded-Chunked,Claded,Chunked,338.466667,784.0,9.29
8,8,Claded-Chunked,Claded,Chunked,307.666667,803.0,8.89
9,9,Claded-Chunked,Claded,Chunked,277.050000,828.0,8.58


In [16]:
q = ! grep kf2d-v1-0-59-10ktol-k7-claded-chunked-rt-cl2- 128G/Memory*csv 
[w.split(",")[-1] for w in q]

['2.49 GiB',
 '3.09 GiB',
 '7.06 GiB',
 '7.06 GiB',
 '9.62 GiB',
 '7.87 GiB',
 '7.87 GiB',
 '7.88 GiB',
 '7.81 GiB',
 '7.76 GiB',
 '7.89 GiB',
 '7.90 GiB',
 '7.90 GiB',
 '7.90 GiB',
 '7.89 GiB',
 '7.90 GiB',
 '7.90 GiB',
 '7.80 GiB',
 '7.90 GiB',
 '7.90 GiB',
 '7.91 GiB',
 '7.82 GiB',
 '7.91 GiB',
 '7.92 GiB',
 '7.91 GiB',
 '7.92 GiB',
 '7.92 GiB',
 '7.82 GiB',
 '7.87 GiB',
 '7.85 GiB',
 '7.92 GiB',
 '7.95 GiB',
 '7.95 GiB',
 '7.96 GiB',
 '7.95 GiB',
 '7.89 GiB',
 '7.95 GiB',
 '7.87 GiB',
 '7.98 GiB',
 '8.00 GiB',
 '8.00 GiB',
 '8.01 GiB',
 '8.01 GiB',
 '7.98 GiB',
 '8.01 GiB',
 '8.01 GiB',
 '7.98 GiB',
 '7.98 GiB',
 '8.01 GiB',
 '2.23 GiB',
 '3.92 GiB',
 '5.37 GiB',
 '6.53 GiB',
 '7.94 GiB',
 '7.88 GiB',
 '7.83 GiB',
 '7.88 GiB',
 '7.88 GiB',
 '7.80 GiB',
 '7.90 GiB',
 '7.92 GiB',
 '7.92 GiB',
 '7.95 GiB',
 '8.01 GiB',
 '8.01 GiB',
 '7.91 GiB',
 '7.91 GiB',
 '7.90 GiB',
 '7.93 GiB',
 '7.93 GiB',
 '7.93 GiB',
 '8.00 GiB',
 '8.01 GiB',
 '8.00 GiB']

In [189]:
df_tmp.to_csv(os.path.join(os.getcwd(), "runtime_mem_claded_chunked.csv"), index=False, sep=' ', header=True)

In [17]:
df_tmp["time"].mean()

270.79

In [79]:
classes_p = "k7_v40_8k_s28All_HiFi_Reads_p_all_ChunkModel"
classes_f = "k7_v40_8k_s28All_HiFi_Reads_p_all_ChunkModel_classes_w_header.out"
classes_df = pd.read_csv(os.path.join(classes_p, classes_f), sep = "\t", header = 0)

In [80]:
#classes_df = classes_df.loc[:, classes_df.columns.intersection(['genome','top_class'])]

In [81]:
classes_df.rename(columns=lambda x: 'clade_' + x if x not in ['genome','top_class', 'top_p'] else x, inplace=True)


In [82]:
classes_df

,genome,top_class,top_p,clade_0,clade_1,clade_2,clade_3,clade_4,clade_5,clade_6,clade_7,clade_8,clade_9,clade_10,clade_11,clade_12,clade_13,clade_14
0,G002003465.part_33_6,2.0,0.998326,1.440292e-14,2.672434e-16,9.983261e-01,2.569205e-14,1.623259e-12,1.622618e-03,6.516443e-15,8.359105e-09,3.960605e-21,5.124596e-05,1.223890e-14,5.247754e-12,2.044956e-18,6.455131e-16,4.495788e-12
1,G900097105.part_1_240,11.0,0.994392,6.542613e-10,2.421036e-03,2.595477e-10,3.649831e-07,8.962098e-08,1.980836e-03,1.040672e-03,1.388056e-10,1.549284e-17,9.295422e-05,1.482703e-05,9.943920e-01,5.669647e-05,2.208454e-07,4.014692e-07
2,G900104395.part_49_2,14.0,1.000000,5.500738e-14,1.369087e-23,1.518141e-07,7.492356e-08,9.510588e-14,3.177374e-12,2.542304e-08,3.188816e-21,3.676304e-20,8.453979e-20,3.671062e-09,1.083858e-10,1.181866e-09,3.461074e-08,9.999998e-01
3,G900100715.part_13_34,13.0,1.000000,1.470343e-20,1.162537e-15,4.065682e-20,3.623664e-10,4.426537e-15,2.055586e-17,4.208101e-08,4.291271e-19,1.561052e-19,4.392510e-14,2.685285e-08,4.138765e-10,8.163479e-15,1.000000e+00,4.538152e-15
4,G900100715.part_13_21,13.0,0.999797,3.498005e-16,7.870918e-16,2.892532e-18,9.026287e-09,3.964643e-13,5.749067e-17,1.327709e-05,3.300685e-15,1.537542e-20,1.197693e-15,2.828005e-06,1.408656e-06,4.688300e-16,9.997973e-01,1.851407e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102423,G001610875.part_1_322,11.0,0.995601,3.425278e-14,5.211755e-16,1.344175e-22,4.652473e-23,2.293646e-03,3.018831e-06,6.929324e-19,9.411742e-16,2.886510e-13,1.735658e-10,1.284666e-03,9.956012e-01,2.974521e-17,6.896235e-10,8.174576e-04
102424,G001603165.part_76_1,10.0,0.918928,1.469179e-06,1.373874e-08,3.040122e-20,5.139501e-06,2.928129e-04,1.438446e-12,1.124979e-06,2.812891e-10,5.398011e-13,5.285415e-08,9.189284e-01,2.892018e-03,6.992103e-13,2.662256e-11,7.787896e-02
102425,G001603165.part_24_4,14.0,0.996516,4.403235e-12,8.457097e-15,7.546505e-21,6.647825e-10,9.432021e-07,1.240444e-08,3.199608e-12,2.575655e-19,5.054411e-13,3.343147e-12,7.203392e-04,2.762999e-03,1.184591e-11,5.682856e-14,9.965156e-01
102426,G001644275.part_112_5,4.0,0.973409,2.586046e-08,5.306511e-06,1.621907e-11,2.796406e-16,9.734085e-01,4.426077e-09,1.041334e-12,3.821646e-11,2.998727e-12,7.021781e-05,1.675721e-06,2.651420e-02,3.359450e-14,3.016585e-09,8.889216e-12


In [83]:
classes_df["tmp"] =  classes_df["genome"]
classes_df[['parent_genome', 'suffix']] = classes_df['tmp'].str.split('.', n=1, expand=True)

In [84]:
classes_df.drop(columns = ["tmp", "suffix"], inplace = True)

In [85]:
my_path = "/Users/nora/Documents/ml_metagenomics"

true_clades = pd.read_csv(os.path.join(my_path, "clade_targets.txt"), sep = " ", header = 0)

In [86]:
true_clades

,genome,clade
0,G001940755,13
1,G001918745,13
2,G001920395,13
3,G001918475,13
4,G000724145,13
...,...,...
10570,G001652645,9
10571,G001652685,9
10572,G900061685,9
10573,G900061725,9


In [87]:
classes_df_merged = pd.merge(classes_df, true_clades, how = "left", left_on = "parent_genome", right_on = "genome")

In [88]:
classes_df_merged.drop(columns = ["genome_y"], inplace = True)

In [89]:
classes_df_merged.rename(columns = {"clade":"true_clade"}, inplace = True)

In [90]:
classes_df_merged

,genome_x,top_class,top_p,clade_0,clade_1,clade_2,clade_3,clade_4,clade_5,clade_6,clade_7,clade_8,clade_9,clade_10,clade_11,clade_12,clade_13,clade_14,parent_genome,true_clade
0,G002003465.part_33_6,2.0,0.998326,1.440292e-14,2.672434e-16,9.983261e-01,2.569205e-14,1.623259e-12,1.622618e-03,6.516443e-15,8.359105e-09,3.960605e-21,5.124596e-05,1.223890e-14,5.247754e-12,2.044956e-18,6.455131e-16,4.495788e-12,G002003465,5
1,G900097105.part_1_240,11.0,0.994392,6.542613e-10,2.421036e-03,2.595477e-10,3.649831e-07,8.962098e-08,1.980836e-03,1.040672e-03,1.388056e-10,1.549284e-17,9.295422e-05,1.482703e-05,9.943920e-01,5.669647e-05,2.208454e-07,4.014692e-07,G900097105,11
2,G900104395.part_49_2,14.0,1.000000,5.500738e-14,1.369087e-23,1.518141e-07,7.492356e-08,9.510588e-14,3.177374e-12,2.542304e-08,3.188816e-21,3.676304e-20,8.453979e-20,3.671062e-09,1.083858e-10,1.181866e-09,3.461074e-08,9.999998e-01,G900104395,14
3,G900100715.part_13_34,13.0,1.000000,1.470343e-20,1.162537e-15,4.065682e-20,3.623664e-10,4.426537e-15,2.055586e-17,4.208101e-08,4.291271e-19,1.561052e-19,4.392510e-14,2.685285e-08,4.138765e-10,8.163479e-15,1.000000e+00,4.538152e-15,G900100715,13
4,G900100715.part_13_21,13.0,0.999797,3.498005e-16,7.870918e-16,2.892532e-18,9.026287e-09,3.964643e-13,5.749067e-17,1.327709e-05,3.300685e-15,1.537542e-20,1.197693e-15,2.828005e-06,1.408656e-06,4.688300e-16,9.997973e-01,1.851407e-04,G900100715,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102423,G001610875.part_1_322,11.0,0.995601,3.425278e-14,5.211755e-16,1.344175e-22,4.652473e-23,2.293646e-03,3.018831e-06,6.929324e-19,9.411742e-16,2.886510e-13,1.735658e-10,1.284666e-03,9.956012e-01,2.974521e-17,6.896235e-10,8.174576e-04,G001610875,11
102424,G001603165.part_76_1,10.0,0.918928,1.469179e-06,1.373874e-08,3.040122e-20,5.139501e-06,2.928129e-04,1.438446e-12,1.124979e-06,2.812891e-10,5.398011e-13,5.285415e-08,9.189284e-01,2.892018e-03,6.992103e-13,2.662256e-11,7.787896e-02,G001603165,14
102425,G001603165.part_24_4,14.0,0.996516,4.403235e-12,8.457097e-15,7.546505e-21,6.647825e-10,9.432021e-07,1.240444e-08,3.199608e-12,2.575655e-19,5.054411e-13,3.343147e-12,7.203392e-04,2.762999e-03,1.184591e-11,5.682856e-14,9.965156e-01,G001603165,14
102426,G001644275.part_112_5,4.0,0.973409,2.586046e-08,5.306511e-06,1.621907e-11,2.796406e-16,9.734085e-01,4.426077e-09,1.041334e-12,3.821646e-11,2.998727e-12,7.021781e-05,1.675721e-06,2.651420e-02,3.359450e-14,3.016585e-09,8.889216e-12,G001644275,4


In [91]:
classes_df_merged['top_class'] = classes_df_merged['top_class'].astype('int')
classes_df_merged['true_clade'] = classes_df_merged['true_clade'].astype('int')

In [92]:
len(classes_df_merged)
classes_df_merged.head(n=5)

,genome_x,top_class,top_p,clade_0,clade_1,clade_2,clade_3,clade_4,clade_5,clade_6,clade_7,clade_8,clade_9,clade_10,clade_11,clade_12,clade_13,clade_14,parent_genome,true_clade
0,G002003465.part_33_6,2,0.998326,1.440292e-14,2.672434e-16,9.983261e-01,2.569205e-14,1.623259e-12,1.622618e-03,6.516443e-15,8.359105e-09,3.960605e-21,5.124596e-05,1.223890e-14,5.247754e-12,2.044956e-18,6.455131e-16,4.495788e-12,G002003465,5
1,G900097105.part_1_240,11,0.994392,6.542613e-10,2.421036e-03,2.595477e-10,3.649831e-07,8.962098e-08,1.980836e-03,1.040672e-03,1.388056e-10,1.549284e-17,9.295422e-05,1.482703e-05,9.943920e-01,5.669647e-05,2.208454e-07,4.014692e-07,G900097105,11
2,G900104395.part_49_2,14,1.000000,5.500738e-14,1.369087e-23,1.518141e-07,7.492356e-08,9.510588e-14,3.177374e-12,2.542304e-08,3.188816e-21,3.676304e-20,8.453979e-20,3.671062e-09,1.083858e-10,1.181866e-09,3.461074e-08,9.999998e-01,G900104395,14
3,G900100715.part_13_34,13,1.000000,1.470343e-20,1.162537e-15,4.065682e-20,3.623664e-10,4.426537e-15,2.055586e-17,4.208101e-08,4.291271e-19,1.561052e-19,4.392510e-14,2.685285e-08,4.138765e-10,8.163479e-15,1.000000e+00,4.538152e-15,G900100715,13
4,G900100715.part_13_21,13,0.999797,3.498005e-16,7.870918e-16,2.892532e-18,9.026287e-09,3.964643e-13,5.749067e-17,1.327709e-05,3.300685e-15,1.537542e-20,1.197693e-15,2.828005e-06,1.408656e-06,4.688300e-16,9.997973e-01,1.851407e-04,G900100715,13


In [93]:
my_subset = classes_df_merged.loc[~(classes_df_merged['top_class'] == classes_df_merged['true_clade'])]

In [94]:
my_subset

,genome_x,top_class,top_p,clade_0,clade_1,clade_2,clade_3,clade_4,clade_5,clade_6,clade_7,clade_8,clade_9,clade_10,clade_11,clade_12,clade_13,clade_14,parent_genome,true_clade
0,G002003465.part_33_6,2,0.998326,1.440292e-14,2.672434e-16,9.983261e-01,2.569205e-14,1.623259e-12,1.622618e-03,6.516443e-15,8.359105e-09,3.960605e-21,5.124596e-05,1.223890e-14,5.247754e-12,2.044956e-18,6.455131e-16,4.495788e-12,G002003465,5
5,G900097105.part_1_255,6,0.999986,1.505457e-10,1.493134e-14,3.551766e-22,2.823760e-06,7.012739e-09,1.079211e-05,9.999862e-01,1.692852e-19,1.027342e-20,1.146979e-09,4.963599e-11,1.116779e-07,3.680712e-08,1.253316e-14,7.646001e-09,G900097105,11
13,G900129715.part_14_10,6,0.930587,2.213102e-12,4.323319e-04,7.389141e-15,6.440534e-02,6.859413e-06,9.299332e-07,9.305866e-01,1.283683e-14,2.035455e-16,7.320483e-05,7.614240e-04,3.731827e-03,9.787565e-15,3.420765e-11,1.563062e-06,G900129715,11
21,G001894865.part_35_1,1,0.979836,6.055541e-19,9.798359e-01,1.034386e-19,2.559329e-26,2.016412e-02,1.331923e-18,2.721877e-19,3.460182e-12,5.815117e-28,1.360267e-13,1.028883e-14,2.130423e-16,5.331281e-30,1.630695e-15,5.634645e-17,G001894865,4
38,G001918475.part_75_2,10,0.771912,1.817523e-08,4.976449e-09,1.838878e-07,5.135268e-04,2.363770e-08,5.369083e-06,4.254778e-10,8.222488e-07,7.406211e-12,7.864553e-09,7.719117e-01,1.345601e-05,5.631174e-10,2.267958e-01,7.590676e-04,G001918475,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102408,G001804315.part_72_1,10,0.984118,1.021414e-09,1.125776e-13,1.233024e-21,3.738786e-09,7.011617e-03,5.755278e-13,1.521311e-08,3.205636e-09,2.223256e-16,1.793718e-06,9.841182e-01,8.865653e-03,3.123420e-09,1.537112e-10,2.690779e-06,G001804315,11
102415,G001798315.part_68_2,0,0.831297,8.312972e-01,3.242467e-15,8.156577e-13,1.351848e-07,4.014623e-07,2.651490e-13,3.998767e-06,3.878894e-03,5.086095e-08,6.256199e-05,6.693707e-02,9.781802e-02,1.163188e-06,2.180614e-10,3.286730e-07,G001798315,10
102416,G001779955.part_3_2,0,1.000000,1.000000e+00,7.525556e-19,1.541617e-28,1.775613e-24,1.115940e-19,3.728231e-21,2.617133e-27,3.906135e-16,6.604078e-23,5.041294e-18,3.748502e-18,1.172357e-13,1.013436e-10,2.594326e-23,1.316618e-15,G001779955,12
102421,G001751165.part_93_1,14,0.882128,1.697505e-15,2.891271e-13,3.135253e-11,6.631327e-07,8.573656e-09,2.493577e-13,1.875952e-14,5.974361e-02,5.896051e-18,5.941753e-06,5.808714e-02,3.195795e-05,1.903793e-12,2.429595e-06,8.821282e-01,G001751165,10


In [95]:
len(my_subset)

9897

In [96]:

col_names = ["genome_x", "length", "A", "C", "G", "T", "2", "3", "4", "CpG", "tv", "ts", "CpG-ts"]

length_df = pd.read_csv(os.path.join(os.getcwd(), "10k_tol_queries_HIFI_reads_LENGTH.txt"), sep = "\t", header = None, names = col_names)

In [97]:
length_df

,genome_x,length,A,C,G,T,2,3,4,CpG,tv,ts,CpG-ts
0,G000006865.part_1_100,17541,5303,3503,2728,6007,0,0,0,880,0,0,0
1,G000006865.part_1_102,17377,5071,3550,2964,5792,0,0,0,954,0,0,0
2,G000006865.part_1_103,7702,2325,1531,1298,2548,0,0,0,410,0,0,0
3,G000006865.part_1_104,12745,4215,2401,1675,4454,0,0,0,490,0,0,0
4,G000006865.part_1_110,7828,2571,1261,1492,2504,0,0,0,386,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102423,G900142495.part_8_6,9374,2569,2502,2104,2199,0,0,0,1360,0,0,0
102424,G900142495.part_8_8,9222,2330,2412,2425,2055,0,0,0,1566,0,0,0
102425,G900142495.part_9_11,12460,3151,3559,2739,3011,0,0,0,1856,0,0,0
102426,G900142495.part_9_4,13564,3134,3100,3811,3519,0,0,0,2110,0,0,0


In [98]:
classes_df_merged_2 = pd.merge(classes_df_merged, length_df, how = "left", left_on = "genome_x", right_on = "genome_x" )

In [99]:
classes_df_merged_2

,genome_x,top_class,top_p,clade_0,clade_1,clade_2,clade_3,clade_4,clade_5,clade_6,...,C,G,T,2,3,4,CpG,tv,ts,CpG-ts
0,G002003465.part_33_6,2,0.998326,1.440292e-14,2.672434e-16,9.983261e-01,2.569205e-14,1.623259e-12,1.622618e-03,6.516443e-15,...,1968,2045,1528,0,0,0,1700,0,0,0
1,G900097105.part_1_240,11,0.994392,6.542613e-10,2.421036e-03,2.595477e-10,3.649831e-07,8.962098e-08,1.980836e-03,1.040672e-03,...,6758,9759,6786,0,0,0,5160,0,0,0
2,G900104395.part_49_2,14,1.000000,5.500738e-14,1.369087e-23,1.518141e-07,7.492356e-08,9.510588e-14,3.177374e-12,2.542304e-08,...,2223,3033,4501,0,0,0,536,0,0,0
3,G900100715.part_13_34,13,1.000000,1.470343e-20,1.162537e-15,4.065682e-20,3.623664e-10,4.426537e-15,2.055586e-17,4.208101e-08,...,2640,2628,3500,0,0,0,732,0,0,0
4,G900100715.part_13_21,13,0.999797,3.498005e-16,7.870918e-16,2.892532e-18,9.026287e-09,3.964643e-13,5.749067e-17,1.327709e-05,...,1525,1737,2176,0,0,0,416,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102423,G001610875.part_1_322,11,0.995601,3.425278e-14,5.211755e-16,1.344175e-22,4.652473e-23,2.293646e-03,3.018831e-06,6.929324e-19,...,4396,4796,3585,0,0,0,3128,0,0,0
102424,G001603165.part_76_1,10,0.918928,1.469179e-06,1.373874e-08,3.040122e-20,5.139501e-06,2.928129e-04,1.438446e-12,1.124979e-06,...,3240,2796,2003,0,0,4,1974,0,0,0
102425,G001603165.part_24_4,14,0.996516,4.403235e-12,8.457097e-15,7.546505e-21,6.647825e-10,9.432021e-07,1.240444e-08,3.199608e-12,...,1565,1625,1159,0,0,0,948,0,0,0
102426,G001644275.part_112_5,4,0.973409,2.586046e-08,5.306511e-06,1.621907e-11,2.796406e-16,9.734085e-01,4.426077e-09,1.041334e-12,...,9084,9572,5009,0,0,0,7780,0,0,0


In [100]:
classes_df_merged_2["length_adj"] = classes_df_merged_2["A"] + classes_df_merged_2["C"] + classes_df_merged_2["G"] + classes_df_merged_2["T"]


In [101]:
classes_df_merged_2

,genome_x,top_class,top_p,clade_0,clade_1,clade_2,clade_3,clade_4,clade_5,clade_6,...,G,T,2,3,4,CpG,tv,ts,CpG-ts,length_adj
0,G002003465.part_33_6,2,0.998326,1.440292e-14,2.672434e-16,9.983261e-01,2.569205e-14,1.623259e-12,1.622618e-03,6.516443e-15,...,2045,1528,0,0,0,1700,0,0,0,7230
1,G900097105.part_1_240,11,0.994392,6.542613e-10,2.421036e-03,2.595477e-10,3.649831e-07,8.962098e-08,1.980836e-03,1.040672e-03,...,9759,6786,0,0,0,5160,0,0,0,29328
2,G900104395.part_49_2,14,1.000000,5.500738e-14,1.369087e-23,1.518141e-07,7.492356e-08,9.510588e-14,3.177374e-12,2.542304e-08,...,3033,4501,0,0,0,536,0,0,0,15549
3,G900100715.part_13_34,13,1.000000,1.470343e-20,1.162537e-15,4.065682e-20,3.623664e-10,4.426537e-15,2.055586e-17,4.208101e-08,...,2628,3500,0,0,0,732,0,0,0,12557
4,G900100715.part_13_21,13,0.999797,3.498005e-16,7.870918e-16,2.892532e-18,9.026287e-09,3.964643e-13,5.749067e-17,1.327709e-05,...,1737,2176,0,0,0,416,0,0,0,7759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102423,G001610875.part_1_322,11,0.995601,3.425278e-14,5.211755e-16,1.344175e-22,4.652473e-23,2.293646e-03,3.018831e-06,6.929324e-19,...,4796,3585,0,0,0,3128,0,0,0,16430
102424,G001603165.part_76_1,10,0.918928,1.469179e-06,1.373874e-08,3.040122e-20,5.139501e-06,2.928129e-04,1.438446e-12,1.124979e-06,...,2796,2003,0,0,4,1974,0,0,0,10080
102425,G001603165.part_24_4,14,0.996516,4.403235e-12,8.457097e-15,7.546505e-21,6.647825e-10,9.432021e-07,1.240444e-08,3.199608e-12,...,1625,1159,0,0,0,948,0,0,0,5469
102426,G001644275.part_112_5,4,0.973409,2.586046e-08,5.306511e-06,1.621907e-11,2.796406e-16,9.734085e-01,4.426077e-09,1.041334e-12,...,9572,5009,0,0,0,7780,0,0,0,28663


In [102]:
pl_f = "k7_v40_8k_s28All_HiFi_Reads_p_all_ChunkModel_all.pl_err"
placement_df = pd.read_csv(os.path.join(classes_p, pl_f), sep = " ", header = None, names = ["genome_x", "a", "pl_err", "b"])

In [103]:
# Remove extra rows
placement_df = placement_df[~(placement_df['pl_err'].isna())]

# Remove NAN columns
placement_df.dropna(axis=1, how='all', inplace = True)

In [104]:
placement_df.index.names = ["genome_x"]
placement_df.reset_index(inplace=True)


# Reformat genome name
placement_df['genome_x'] = placement_df['genome_x'].str.replace('apples_input_di_mtrx_query_','')
placement_df['genome_x'] = placement_df['genome_x'].str.replace('.pl_err','')

In [105]:
placement_df

,genome_x,a,pl_err,b
0,G001824435.part_26_2,10069,0,0.0000
1,G001824435.part_26_5,10069,27,0.0027
2,G001824435.part_26_9,10069,0,0.0000
3,G001824435.part_28_1,10069,5,0.0005
4,G001824435.part_28_4,10069,5,0.0005
...,...,...,...,...
102423,G001824435.part_25_5,10069,8,0.0008
102424,G001824435.part_25_6,10069,0,0.0000
102425,G001824435.part_25_9,10069,1,0.0001
102426,G001824435.part_26_11,10069,0,0.0000


In [106]:
df_summary = pd.merge(placement_df, classes_df_merged_2, how = 'left', left_on = "genome_x", right_on = "genome_x")

In [107]:
df_summary

,genome_x,a,pl_err,b,top_class,top_p,clade_0,clade_1,clade_2,clade_3,...,G,T,2,3,4,CpG,tv,ts,CpG-ts,length_adj
0,G001824435.part_26_2,10069,0,0.0000,0,0.999997,0.999997,6.282689e-17,2.060929e-21,1.476329e-19,...,2141,2031,0,0,0,1368,0,0,0,7902
1,G001824435.part_26_5,10069,27,0.0027,14,0.934256,0.052527,1.890353e-09,2.537158e-13,6.216167e-03,...,1164,1445,0,0,0,840,0,0,0,5472
2,G001824435.part_26_9,10069,0,0.0000,0,1.000000,1.000000,4.667032e-23,1.538975e-25,1.393644e-20,...,1124,1420,0,0,0,842,0,0,0,5339
3,G001824435.part_28_1,10069,5,0.0005,0,1.000000,1.000000,7.948286e-22,7.019722e-21,3.158791e-21,...,2742,3013,0,0,0,1642,0,0,0,11136
4,G001824435.part_28_4,10069,5,0.0005,0,1.000000,1.000000,3.048809e-20,1.115820e-20,1.020783e-21,...,2133,2610,0,0,35,1262,0,0,0,9176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102423,G001824435.part_25_5,10069,8,0.0008,0,1.000000,1.000000,6.982934e-13,5.984320e-15,9.183989e-13,...,2027,2696,0,0,0,1346,0,0,0,9764
102424,G001824435.part_25_6,10069,0,0.0000,0,0.999999,0.999999,4.403461e-16,5.082139e-23,1.013607e-15,...,1389,1452,0,0,0,834,0,0,0,5586
102425,G001824435.part_25_9,10069,1,0.0001,0,0.895721,0.895721,7.342496e-08,6.008305e-18,2.317667e-11,...,1149,1844,0,0,0,650,0,0,0,5659
102426,G001824435.part_26_11,10069,0,0.0000,0,0.999991,0.999991,3.321387e-22,2.460953e-27,1.987862e-14,...,1336,1222,0,0,0,806,0,0,0,5071


In [108]:
df_summary["model"] = "chunk"

In [109]:
df_summary.to_csv(os.path.join(os.getcwd(), "summary_hifi_chunk_model.pl_error"), index=False, sep=' ', header=True)

In [65]:
des = df_summary.groupby("parent_genome").describe()
#des = df_summary.groupby(["parent_genome"])['pl_err', ].mean()

In [129]:
des.reset_index(inplace=True)

In [130]:
des

parent_genome      a                                                    \
                   count     mean  std      min      25%      50%      75%   
0      G000006865  176.0  10069.0  0.0  10069.0  10069.0  10069.0  10069.0   
1      G000006925  373.0  10069.0  0.0  10069.0  10069.0  10069.0  10069.0   
2      G000007385  359.0  10069.0  0.0  10069.0  10069.0  10069.0  10069.0   
3      G000007705  356.0  10069.0  0.0  10069.0  10069.0  10069.0  10069.0   
4      G000007845  395.0  10069.0  0.0  10069.0  10069.0  10069.0  10069.0   
..            ...    ...      ...  ...      ...      ...      ...      ...   
495    G900129715  286.0  10069.0  0.0  10069.0  10069.0  10069.0  10069.0   
496    G900129805  159.0  10069.0  0.0  10069.0  10069.0  10069.0  10069.0   
497    G900129845  243.0  10069.0  0.0  10069.0  10069.0  10069.0  10069.0   
498    G900130035  245.0  10069.0  0.0  10069.0  10069.0  10069.0  10069.0   
499    G900142495  281.0  10069.0  0.0  10069.0  10069.0  10069.0  10069.0   

             pl_err  ... CpG-ts      length_adj                             \
         max  count  ...    75%  max      count          mean          std   
0    10069.0  176.0  ...    0.0  0.0      176.0  12126.482955  7034.067522   
1    10069.0  373.0  ...    0.0  0.0      373.0  11567.900804  6173.670073   
2    10069.0  359.0  ...    0.0  0.0      359.0  12126.715877  6417.875431   
3    10069.0  356.0  ...    0.0  0.0      356.0  12206.918539  6952.165247   
4    10069.0  395.0  ...    0.0  0.0      395.0  12110.670886  6767.358949   
..       ...    ...  ...    ...  ...        ...           ...          ...   
495  10069.0  286.0  ...    0.0  0.0      286.0  11353.080420  5873.325069   
496  10069.0  159.0  ...    0.0  0.0      159.0  11198.836478  5756.736091   
497  10069.0  243.0  ...    0.0  0.0      243.0  11743.234568  6443.362254   
498  10069.0  245.0  ...    0.0  0.0      245.0  11630.297959  6379.070461   
499  10069.0  281.0  ...    0.0  0.0      281.0  11675.234875  6391.916673   

                                                  
        min      25%      50%       75%      max  
0    5084.0  7095.50  10439.0  14213.00  45955.0  
1    5063.0  6829.00   9654.0  15071.00  45955.0  
2    5134.0  7345.50   9982.0  15269.00  45955.0  
3    5007.0  7067.50  10331.0  15625.75  45955.0  
4    5007.0  7097.00  10144.0  15610.50  45955.0  
..      ...      ...      ...       ...      ...  
495  5020.0  6961.75   9832.0  13339.25  41230.0  
496  5094.0  6943.00   9644.0  13361.50  44811.0  
497  5007.0  7154.50   9651.0  14759.50  41073.0  
498  5012.0  7021.00  10044.0  13921.00  45955.0  
499  5009.0  7301.00   9845.0  13996.00  48816.0  

[500 rows x 145 columns]

In [131]:
des.columns.to_list()

[('parent_genome', ''),
 ('a', 'count'),
 ('a', 'mean'),
 ('a', 'std'),
 ('a', 'min'),
 ('a', '25%'),
 ('a', '50%'),
 ('a', '75%'),
 ('a', 'max'),
 ('pl_err', 'count'),
 ('pl_err', 'mean'),
 ('pl_err', 'std'),
 ('pl_err', 'min'),
 ('pl_err', '25%'),
 ('pl_err', '50%'),
 ('pl_err', '75%'),
 ('pl_err', 'max'),
 ('b', 'count'),
 ('b', 'mean'),
 ('b', 'std'),
 ('b', 'min'),
 ('b', '25%'),
 ('b', '50%'),
 ('b', '75%'),
 ('b', 'max'),
 ('top_class', 'count'),
 ('top_class', 'mean'),
 ('top_class', 'std'),
 ('top_class', 'min'),
 ('top_class', '25%'),
 ('top_class', '50%'),
 ('top_class', '75%'),
 ('top_class', 'max'),
 ('true_clade', 'count'),
 ('true_clade', 'mean'),
 ('true_clade', 'std'),
 ('true_clade', 'min'),
 ('true_clade', '25%'),
 ('true_clade', '50%'),
 ('true_clade', '75%'),
 ('true_clade', 'max'),
 ('length', 'count'),
 ('length', 'mean'),
 ('length', 'std'),
 ('length', 'min'),
 ('length', '25%'),
 ('length', '50%'),
 ('length', '75%'),
 ('length', 'max'),
 ('A', 'count'),
 ('A'

In [132]:
des['pl_err', 'mean']

0      14.829545
1      20.402145
2      12.395543
3      18.258427
4      13.336709
         ...    
495    14.479021
496     7.289308
497    12.790123
498     5.848980
499     9.014235
Name: (pl_err, mean), Length: 500, dtype: float64

In [133]:
final_table_columns = [('parent_genome',      ''), ('pl_err', 'mean'), ('pl_err', 'count')]
des2 = des.drop(columns=[col for col in des if col not in final_table_columns])

In [134]:
des2

parent_genome pl_err           
                   count       mean
0      G000006865  176.0  14.829545
1      G000006925  373.0  20.402145
2      G000007385  359.0  12.395543
3      G000007705  356.0  18.258427
4      G000007845  395.0  13.336709
..            ...    ...        ...
495    G900129715  286.0  14.479021
496    G900129805  159.0   7.289308
497    G900129845  243.0  12.790123
498    G900130035  245.0   5.848980
499    G900142495  281.0   9.014235

[500 rows x 3 columns]